In [ ]:
import pandas as pd
import torch
import os, sys, glob
from utils import ProgressMeter, AverageMeter, save_checkpoint, TiffDataset


In [ ]:
cores_chemo_labels_df = pd.read_csv('data/cores_labels_chemotherapy.csv')

In [ ]:
cores_path = "/data/projects/pixel_project/datasets/NKI_project_TMAs/"
cores_files = []
cores_directories = [d for d in os.listdir(cores_path) if
                     os.path.isdir(os.path.join(cores_path, d)) and d.startswith('TMA')]
cores_files_train = []
cores_files_test = []
cores_labels_train = []
cores_labels_test = []
input_dimensions = (1024, 1024)
batch_size = 8
num_workers = 28
for i, slide in enumerate(cores_directories):
    files_path = str(cores_path) + "/" + slide + "/Channels_all"
    cores_files.extend([os.path.join(r, fn)
                        for r, ds, fs in os.walk(files_path)
                        for fn in fs if fn.endswith('.tif')])

for i, core_file in enumerate(cores_files):
    patch_file_label_df = cores_chemo_labels_df[(cores_chemo_labels_df['cycif.slide']==core_file.split('/')[-3])&(cores_chemo_labels_df['cycif.core.id']==core_file.split('/')[-1].replace('.tif',''))]
    if not patch_file_label_df.empty and str(patch_file_label_df.iloc[0]['therapy_sequence']).lower()!='na':
        if core_file.split('/')[-3]=='TMA_42_961':
            cores_files_test.append(core_file)
            # If contains NACT, is a sample collected after chemotherapy exposure
            if 'nact' in str(patch_file_label_df.iloc[0]['therapy_sequence']).lower():
                cores_labels_test.append(0)
            else:
                cores_labels_test.append(1)
        else:
            cores_files_train.append(core_file)
            # If contains NACT, is a sample collected after chemotherapy exposure
            if 'nact' in str(patch_file_label_df.iloc[0]['therapy_sequence']).lower():
                cores_labels_train.append(0)
            else:
                cores_labels_train.append(1)
    else:
        print('Missing label for:'+core_file)

tiff_dataset_train = TiffDataset(files=cores_files_train,transform=T.Resize([input_dimensions[0],input_dimensions[1]]), channels=channels,labels=cores_labels_train)
tiff_dataset_test = TiffDataset(files=cores_files_test,transform=T.Resize([input_dimensions[0],input_dimensions[1]]), channels=channels,labels=cores_labels_test)

train_sampler = None
train_loader = torch.utils.data.DataLoader(
        tiff_dataset_train, batch_size=batch_size, shuffle=(train_sampler is None),
         pin_memory=True, sampler=train_sampler, num_workers=num_workers)
test_sampler = None
test_loader = torch.utils.data.DataLoader(
        tiff_dataset_test, batch_size=batch_size, shuffle=(test_sampler is None),
         pin_memory=True, sampler=test_sampler, num_workers=num_workers)
